<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [2]:
import requests
import os
from google.colab import output
import pprint
from io import BytesIO
import PyPDF2
import re
import datetime
import dask

In [3]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [4]:
def list_endpoints():
    date_today = datetime.datetime.now()
    year_today = date_today.year
    month_today = date_today.month
    day_today = date_today.day
    key = "qu2TgXAQPvswtEZ176PFWyfvdb5wd1nwcv2JwEUo"

    endpoints = []
    for i in range(1, 3650):
        past_date = date_today - datetime.timedelta(days=i)
        year = past_date.strftime("%Y")
        month = past_date.strftime("%m")
        day = past_date.strftime("%d")
        endpoint = f"https://api.congress.gov/v3/congressional-record/?y={year}&m={month}&d={day}&api_key={key}"
        endpoints.append(endpoint)
    return endpoints


endpoints_result = list_endpoints()

In [13]:
def find_pdf_links(endpoints_list):
  try:
    links_to_pdfs_with_matches = []
    for i in endpoints_list:
        response = requests.get(i)
        if response.json()['Results']['Issues'] != []:
          link_to_pdf_file = response.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
          pdf_file = requests.get(link_to_pdf_file)
          pdf_data = BytesIO(pdf_file.content)
          pdf_reader = PyPDF2.PdfReader(pdf_data)
          number_of_pages = len(pdf_reader.pages)
          for j in range(number_of_pages):
            page = pdf_reader.pages[j]
            text = page.extract_text()
            adam_smith_matches = re.findall(r'Adam\sSmith\W*', text)
            if adam_smith_matches != []:
              print(adam_smith_matches)
              print(link_to_pdf_file)
              links_to_pdfs_with_matches.append(link_to_pdf_file)
    return links_to_pdfs_with_matches
  except Exception as e:
    print(e)
    return links_to_pdfs_with_matches



delayed_pdf_links = dask.delayed(find_pdf_links)(endpoints_result)

In [15]:
actual_pdf_links = delayed_pdf_links.compute()
actual_pdf_links

['Adam Smith .................................................... ']
https://www.congress.gov/118/crec/2024/03/29/170/54/CREC-2024-03-29.pdf
['Adam Smith, ']
https://www.congress.gov/118/crec/2023/04/19/169/65/CREC-2023-04-19.pdf
['Adam Smith .................................................... ']
https://www.congress.gov/118/crec/2023/04/06/169/60/CREC-2023-04-06.pdf
['Adam Smith’']
https://www.congress.gov/118/crec/2023/03/09/169/45/CREC-2023-03-09.pdf
['Adam Smith ']
https://www.congress.gov/118/crec/2023/01/27/169/18/CREC-2023-01-27.pdf
['Adam Smith, ']
https://www.congress.gov/118/crec/2023/01/27/169/18/CREC-2023-01-27.pdf
['Adam Smith \n(']
https://www.congress.gov/117/crec/2022/12/21/168/199/CREC-2022-12-21.pdf
['Adam Smith, ']
https://www.congress.gov/117/crec/2022/12/08/168/191/CREC-2022-12-08.pdf
['Adam Smith—']
https://www.congress.gov/117/crec/2022/07/26/168/124/CREC-2022-07-26.pdf
['Adam Smith, ']
https://www.congress.gov/117/crec/2022/07/19/168/119/CREC-2022-07-19.pdf
['A

['https://www.congress.gov/118/crec/2024/03/29/170/54/CREC-2024-03-29.pdf',
 'https://www.congress.gov/118/crec/2023/04/19/169/65/CREC-2023-04-19.pdf',
 'https://www.congress.gov/118/crec/2023/04/06/169/60/CREC-2023-04-06.pdf',
 'https://www.congress.gov/118/crec/2023/03/09/169/45/CREC-2023-03-09.pdf',
 'https://www.congress.gov/118/crec/2023/01/27/169/18/CREC-2023-01-27.pdf',
 'https://www.congress.gov/118/crec/2023/01/27/169/18/CREC-2023-01-27.pdf',
 'https://www.congress.gov/117/crec/2022/12/21/168/199/CREC-2022-12-21.pdf',
 'https://www.congress.gov/117/crec/2022/12/08/168/191/CREC-2022-12-08.pdf',
 'https://www.congress.gov/117/crec/2022/07/26/168/124/CREC-2022-07-26.pdf',
 'https://www.congress.gov/117/crec/2022/07/19/168/119/CREC-2022-07-19.pdf',
 'https://www.congress.gov/117/crec/2022/04/07/168/62/CREC-2022-04-07.pdf',
 'https://www.congress.gov/117/crec/2022/02/08/168/25/CREC-2022-02-08.pdf',
 'https://www.congress.gov/117/crec/2021/10/07/167/177/CREC-2021-10-07.pdf',
 'https